scaricare hdf view richiede acco gratuito per i download

+ link manuale prisma obsoleti? https://wiki.earthdata.nasa.gov/display/DAS/Toolkit+Downloads  

+ **prisma usa librerie hdf-eos5 v1.13, hdf-eos5 è ora a versione 1.16**  

+ esempio di mmap su hdf5(non hdf-eos5) con julia https://github.com/JuliaIO/HDF5.jl/blob/master/test/mmap.jl  

+ immagine docker per hdf eos lib https://hub.docker.com/r/hdfeos/hdfeos  

+ sdp toolkit: https://wiki.earthdata.nasa.gov/display/DAS/SDP+Toolkit+and+HDF-EOS+EOSDIS++Core+System+Project  

+ tutorial (in c) https://portal.hdfgroup.org/display/HDF5/Learning+the+Basics

+ hdfview: tool da https://support.hdfgroup.org/ftp/HDF5/releases/HDF-JAVA/hdf-java-2.10.1/bin/ (plugin non sembra funzionare con versione più recente)
  plugin per hdf-eos da https://wiki.earthdata.nasa.gov/display/DAS/HDFView+HDF-EOS+Plug-in  
  guida installazione: https://newsroom.gsfc.nasa.gov/sdptoolkit/HDFView/README_HDFView_plugin_v2.10.html  
  
+ documentazione pkg hdf5 https://juliaio.github.io/HDF5.jl/stable/

+ pkg R per importare dati prisma l1/l2 https://lbusett.github.io/prismaread/

In [3]:
# docu package hdf5 per julia https://juliaio.github.io/HDF5.jl/stable/
# using Pkg
# pkg.add("HDF5")
using HDF5

#errori posson lasciare aperti file-> file inseriti in struct dinamica per poi esser facilmente chiusi tutti
openfiles = Dict()
counter = 0
# funzione che chiude tutti i file aperti
function closeall(dict)
    for (key, value) in dict
        close(pop!(dict, key))    
    end
    counter = 0
    openfiles
end

closeall (generic function with 1 method)

apertura file con h5open(filename::string, mode::string) -> file   
mode:  

| mode | Meaning |
|---|---|
|"r"|read-only|
|"r+"|read-write, preserving any existing contents|
|"cw"|read-write, create file if not existing, preserve existing contents|
|"w"|read-write, destroying any existing contents (if any)|

In [3]:
testfilename = "test.h5"
fid = h5open(testfilename, "w") # crea file hdf5 vuoto (o lo apre e basta se già esiste)

close(fid)


HDF5 data file: test.h5

In [6]:
close(fid)
rm(testfilename) #deleta file di prova

In [4]:
isopen(fid)

true

In [36]:
function viewfile(x)
    println("call viewfile")
   #=for i in x 
        if typeof(i)==HDF5Group
            viewgroup(i)
        else
            println(typeof(i))
        end
    end=#
    viewgroup(x)
end
    
function viewgroup(g)
    println("call viewgroup on $g")
    #nome
    println(g)
    
    # attributi
    
    #view su oggetti contenuti
    for i in g
        ti = typeof(i)
        if ti==HDF5Group
            viewgroup(i)
        elseif ti==HDF5Dataset
            viewdata(i)
        else
            println(ti)
        end
    end
end

function viewdata(d)
    println("call viewdata on $d")
    # nome
    println(d)
    #1 dataset mappabile?
    if ismmappable(d)
        println("dataset mmappabile")
    else
        println("dataset non mmappabile")
    end
    #mappatura dataset con readmmap() aumenta rpestazioni per letture numerose di un ds
    
    # datatype
    
    # dataspace (dimensioni )
    
    # nome
end   
            

viewdata (generic function with 1 method)

In [37]:
function foo(x) 
    for i in x 
        println(typeof(i))
    end
    println(typeof(x))    
end

foo (generic function with 1 method)

In [38]:
#apro in lettura un file sample preso da hdfviewer, faccio qualcosa, chiudo file
#inputfile = "OMI-Aura_.he5"
inputfile = "hdf5_test.h5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
counter = counter+1
#foo(file2)
viewfile(file2)
closeall(openfiles)
isopen(file2)
openfiles

ishdf5 ? true
call viewfile
call viewgroup on HDF5 data file: hdf5_test.h5
HDF5 data file: hdf5_test.h5
call viewdata on HDF5 dataset: /A note (file: hdf5_test.h5 xfer_mode: 0)
dataset non mmappabile
call viewdata on HDF5 dataset: /Notes (file: hdf5_test.h5 xfer_mode: 0)
dataset non mmappabile
call viewgroup on HDF5 group: /arrays (file: hdf5_test.h5)
HDF5 group: /arrays (file: hdf5_test.h5)
call viewdata on HDF5 dataset: /arrays/1D String (file: hdf5_test.h5 xfer_mode: 0)
dataset non mmappabile
call viewdata on HDF5 dataset: /arrays/2D float array (file: hdf5_test.h5 xfer_mode: 0)
dataset mmappabile
call viewdata on HDF5 dataset: /arrays/2D int array (file: hdf5_test.h5 xfer_mode: 0)
dataset mmappabile
call viewdata on HDF5 dataset: /arrays/2D int8x9 (file: hdf5_test.h5 xfer_mode: 0)
dataset mmappabile
call viewdata on HDF5 dataset: /arrays/3D int array (file: hdf5_test.h5 xfer_mode: 0)
dataset mmappabile
call viewdata on HDF5 dataset: /arrays/4D int (file: hdf5_test.h5 xfer_mode: 0)


Dict{Any,Any}()

closeall (generic function with 1 method)

In [37]:
openfiles

Dict{Any,Any}()

In [39]:
?datatype


search: datatype DataType HDF5Datatype



No documentation found.

`HDF5.datatype` is a `Function`.

```
# 12 methods for generic function "datatype":
[1] datatype(str::String) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1187
[2] datatype(dset::HDF5Attribute) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1171
[3] datatype(dset::HDF5Dataset) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1169
[4] datatype(x::Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj}) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1174
[5] datatype(::Type{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1175
[6] datatype(str::Array{S,N} where N) where S<:String in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1193
[7] datatype(A::AbstractArray{T,N} where N) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1176
[8] datatype(::Type{Complex{T}}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1177
[9] datatype(x::Complex{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1184
[10] datatype(A::AbstractArray{Complex{T},N} where N) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1185
[11] datatype(A::HDF5Vlen{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1199
[12] datatype(str::HDF5Vlen{C}) where C<:HDF5.CharType in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1200
```
